## CAI - Rebalancing data for March 2023 

## Configuring packages

In [21]:
import pandas as pd
import numpy as np
import requests
import decouple
import sys 
import datetime
import time
sys.path.append('../')

from web3 import Web3
from abis import index_anatomy


infura_url = decouple.config("AVALANCHE_INFURA_URL")
web3_object = Web3(Web3.HTTPProvider(infura_url))

import db_funcs

In [22]:
from pycoingecko import CoinGeckoAPI
key = decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key= key)

In [23]:
import plotly.express as px
import plotly.graph_objects as go

In [24]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Index Variables and settings

In [25]:
db_key_date = '2023-03-01'
db_liquidity_table = 'cai_liquidities'
db_benchmark_table = 'cai_benchmark_data'
min_weight = 0.005
max_weight = 0.5
supply_threshold = 0.3
liquidity_consistency = 90 # In days
avax_to_savax_split = 0.6
liveness_threshold = 30*6 # In days

In [26]:
liq = 2*10**6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .5
max_slippage = round(2 * max_slippage, 2) / 2 
max_slippage

-0.09

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data at least 6 months prior to the date of inclusion in the index.
    1. The project should have a token that is native to Avalanche. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project.
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible.
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
    <br>
1. **Pricing requirements**
    1. Token should have a Chainlink price feed to allow for manipulation resistant pricing.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange. 
    1. The token should have in aggregate at least $2mm of onchain liquidity across Trader Joe, Platypus, Pangolin.
    1. The token must have shown consistent DeFi liquidity on Avalanche.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [27]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### CoinGecko

##### Tokens addresses

In [28]:
coins_list = cg.get_coins_list(include_platform=True)
coins = {}
for coin in coins_list:
    if 'avalanche' in coin['platforms']:
        coin['addr'] = coin['platforms']['avalanche']
        coin['symbol'] = normalize_symb(coin['symbol'])
        coins[coin['id']] = coin

# Adding AVAX
coins['avalanche-2'] = {'id': 'avalanche-2','symbol':normalize_symb('avax'), 'name':'Avalanche','addr':'0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7',}


##### Market data

In [29]:
cat_id = "avalanche-ecosystem"
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id)
for i in range(2, 30):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id, page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)

cg_tokens_data = cg_tokens_data.append(cg.get_coins_markets(ids='avalanche-2',vs_currency='USD'),ignore_index=False)

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_85955/3118279002.py:8: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [30]:
tokens_market_data = cg_tokens_data.set_index('id')[
    ['name', 'symbol', 'market_cap', 'market_cap_rank', 'current_price', 'total_supply', 'circulating_supply']
]
tokens_market_data['symbol'] = tokens_market_data['symbol'].apply(normalize_symb)

In [31]:
tokens_market_data = tokens_market_data[tokens_market_data['market_cap'] > 0]
tokens_market_data = tokens_market_data[~tokens_market_data.index.duplicated(keep='first')]
tokens_market_data = tokens_market_data.join(pd.DataFrame.from_dict(coins, orient='index')['addr'])
tokens_market_data = tokens_market_data[~tokens_market_data['addr'].isna()]


In [32]:
# Stable coins to remove from tokens list
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')

#Adjusting Pangolin's max supply to reflect tokenomic changes here: https://app.pangolin.exchange/#/vote/10
tokens_market_data.loc[['pangolin'],["total_supply"]] = 230000000.0

tokens_market_data =  tokens_market_data[~tokens_market_data.index.isin(stablecoins.index)]
tokens_market_data = tokens_market_data.sort_values('market_cap', ascending=False)

tokens_market_data

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,6.355086e+09,18.0,20.170000,4.205317e+08,3.151257e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
uniswap,Uniswap,UNI,5.290212e+09,19.0,7.040000,1.000000e+09,7.537667e+08,0x8ebaf22b6f053dffeaf46f4dd9efa95d89ba8580
wrapped-bitcoin,Wrapped Bitcoin,WBTC,4.317859e+09,21.0,24654.000000,1.754971e+05,1.754971e+05,0x50b7545627a5162f82a992c33b87adc75187b218
chainlink,Chainlink,LINK,3.802726e+09,23.0,7.740000,1.000000e+09,4.916000e+08,0x5947bb275c521040051d82396192181b413227a3
the-graph,The Graph,GRT,1.513603e+09,41.0,0.169298,1.000000e+10,8.893377e+09,0x8a0cac13c7da965a312f08ea4229c37869e85cb9
...,...,...,...,...,...,...,...,...
fortressdao,Fortress,FORT,1.196430e+04,3730.0,0.008534,1.409126e+06,1.401945e+06,0xf6d46849db378ae01d93732585bec2c4480d1fd5
morphie,Morphie,MRFI,8.163660e+03,3768.0,0.000001,2.000000e+10,8.069296e+09,0x05e481b19129b560e921e487adb281e70bdba463
hermes,HERMES,HERMES,2.930000e+03,3853.0,0.478798,9.878883e+03,6.119485e+03,0xb15f02f9da8cd1f99e9dd375f21dc96d25ddd82c


### Token's supply check 

In [33]:
sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > supply_threshold
sp_passed = tokens_market_data[sp_passed_index]

### Retrieve historical data & token age check

In [34]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for id_, row in sp_passed.iterrows():
    symbol = row['symbol']
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', id_])
    df_prices = df_prices[df_prices[id_] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < liveness_threshold:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < {liveness_threshold} days')
        exclude_list.append(id_)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', id_])
    df_mcaps = df_mcaps[df_mcaps[id_] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < liveness_threshold:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < {liveness_threshold} days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Note: BTCB, marketcap data available only for 127 < 180 days
Excluding USP, prices data available only for 12 < 180 days
Excluding XEN, prices data available only for 134 < 180 days
Note: CLY, marketcap data available only for 172 < 180 days
Note: BUILD, marketcap data available only for 179 < 180 days
Note: KENSHI, marketcap data available only for 19 < 180 days
Note: 3ULL, marketcap data available only for 151 < 180 days
Note: DCAR, marketcap data available only for 160 < 180 days
Excluding CAI, prices data available only for 168 < 180 days
Note: MAGIC, marketcap data available only for 41 < 180 days
Note: CHRO, marketcap data available only for 106 < 180 days
Note: FRF, marketcap data available only for 14 < 180 days


### Non quantative checks

In [35]:
manual_exclusions = [
    {
        'gecko_id': 'spell-token',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'stacktical',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'zookeeper',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'pendle',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'everrise',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'wrapped-bitcoin',
        'reason' : 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'bitcoin-avalanche-bridged-btc-b',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'gmx',
        'reason': 'Bridged asset'
    },
    {
        'gecko_id': 'woo-network',
        'reason': 'Bridged asset'
    }
]
for token in manual_exclusions:
    exclude_list.append(token['gecko_id'])

In [36]:
passed_q_and_non_q_checks = sp_passed.query('index not in @exclude_list')
passed_q_and_non_q_checks.drop(passed_q_and_non_q_checks[passed_q_and_non_q_checks['market_cap'] < 1e6].index)

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,6.355086e+09,18.0,20.170000,4.205317e+08,3.151257e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
uniswap,Uniswap,UNI,5.290212e+09,19.0,7.040000,1.000000e+09,7.537667e+08,0x8ebaf22b6f053dffeaf46f4dd9efa95d89ba8580
chainlink,Chainlink,LINK,3.802726e+09,23.0,7.740000,1.000000e+09,4.916000e+08,0x5947bb275c521040051d82396192181b413227a3
the-graph,The Graph,GRT,1.513603e+09,41.0,0.169298,1.000000e+10,8.893377e+09,0x8a0cac13c7da965a312f08ea4229c37869e85cb9
aave,Aave,AAVE,1.264487e+09,48.0,88.650000,1.600000e+07,1.427036e+07,0x63a72806098bd3d9520cc43356dd78afe5d386d9
...,...,...,...,...,...,...,...,...
atlas-usv,Atlas USV,USV,1.463521e+06,1869.0,13.930000,2.836992e+05,1.050826e+05,0xb0a8e082e5f8d2a04e74372c1be47737d85a0e73
playa3ull-games,Playa3ull Games,3ULL,1.413647e+06,1888.0,0.000624,4.612403e+09,2.264702e+09,0x9e15f045e44ea5a80e7fbc193a35287712cc5569
okcash,Okcash,OK,1.237498e+06,1975.0,0.014293,1.050000e+08,8.676373e+07,0xd3ac016b1b8c80eeadde4d186a9138c9324e4189


### Liquidity check

To derive a slippage threshold, consider an abstract univ2-like pool with $5m\$$ overall liquidity. 
I.e if pool has reserves $R_1$, $R_2$ and $p_1$ and $p_2$ are the correspondent tokens prices (in USD), then
$$R_1p_1 = R_2p_2 = 2.5m \$$$

Swapping $x$ token1 for token2 will result in the output amount $y$ such that
$$R_1\cdot R_2 = (R_1 + x) \cdot (R_2 - y)$$
$$y = \frac{R_2\cdot x}{R_1 + x}$$
Then the delivery price will be
$$p_d = \frac{y}{x} = \frac{R_2}{R_1 + x}$$
The spot price may be estimated as a delivery price for relatively small input amount $x_0$
$$p_s = \frac{y_0}{x_0} = \frac{R_2}{R_1 + x_0}$$

Then the theoretical slippage for this pool will be
$$slippage = \frac{p_d}{p_s} - 1 = \frac{R_1+x_0}{R_1+x} - 1 = \frac{p_1R_1+p_1x_0}{p_1R_1+p_1x}-1$$

In [37]:
url_0x = "https://avalanche.api.0x.org/swap/v1/quote"

In [38]:
decimals = 18
slippages = []
for id, data in passed_q_and_non_q_checks.iterrows():
    try:
        query = {
            'buyToken': data['addr'],
            'sellToken': 'AVAX',
            'sellAmount': int(trade_value_tiny / tokens_market_data.loc['avalanche-2']['current_price'] * 10 ** decimals),
            'enableSlippageProtection':'true'
        }
        
        # spot price is calculated as a price for 100$ swap
        resp = requests.get(url_0x, params=query)
        swap = resp.json()
        spot_price = float(swap['price'])
        
        query['sellAmount'] = int(trade_value / tokens_market_data.loc['avalanche-2']['current_price'] * 10 ** decimals)
        resp = requests.get(url_0x, params=query)
        swap = resp.json()
        del_price = float(swap['price'])
        
        slippage = del_price / spot_price - 1
        slippages.append({
            'id': id,
            'spot_price': spot_price,
            'delivery_price': del_price,
            'slippage': slippage
        })
        time.sleep(2)
    except KeyError:
        print(id)
        continue
    

havven
cartesi
idia
realfevr
build
kenshi
dragon-crypto-aurum
okcash
spherium
shack
bios
avaxtars
snowball-token
france-rev-finance


In [39]:
slippage

-0.9808256525614277

In [56]:
slippages_df = pd.DataFrame(slippages).sort_values('slippage',ascending=False).reset_index(drop=True)
slippages_df.head(20)

,id,spot_price,delivery_price,slippage
0,benqi-liquid-staked-avax,9.398680e-01,9.399014e-01,0.000035
1,avalanche-2,1.000000e+00,1.000000e+00,0.000000
2,joe,6.839880e+01,6.590713e+01,-0.036429
3,benqi,1.727150e+03,1.633371e+03,-0.054296
4,avalaunch,3.752282e+01,3.424749e+01,-0.087289
5,pangolin,3.555311e+02,3.228157e+02,-0.092018
6,chainlink,2.599823e+00,2.192836e+00,-0.156544
7,yield-yak,3.249525e-02,2.519362e-02,-0.224698
8,vector-finance,1.288267e+02,9.445628e+01,-0.266796
9,aave,2.265189e-01,1.508000e-01,-0.334272


In [41]:
# Writing to database
slippages_to_save = slippages_df[['id','slippage']].head(20)
asset_columns = db_funcs.convert_to_sql_strings(list(slippages_to_save['id']))
slippage_values = list(slippages_to_save['slippage'])
db_funcs.insert_values(db_key_date,asset_columns,slippage_values,db_liquidity_table)


In [42]:
def check_avg_liquidity(date_delta):
    start_date = str(datetime.date.fromisoformat(db_key_date) - datetime.timedelta(days=date_delta))
    dataframe = pd.read_sql(f'Select * from {db_liquidity_table} where date >= ? ',db_funcs.create_connection(db_funcs.db),index_col='date',parse_dates=['date'],params=[start_date])
    dataframe = dataframe.mean(axis=0)
    dataframe = dataframe[dataframe > max_slippage]
    dataframe.index = db_funcs.convert_from_sql_strings(list(dataframe.index))
    return dataframe, list(dataframe.index)

avg_liq_dataframe ,include_list = check_avg_liquidity(liquidity_consistency)


In [43]:
include_list = list(slippages_df[abs(slippages_df['slippage']) < abs(max_slippage)]['id'])
qualified_assets = passed_q_and_non_q_checks.query('index in @include_list')
qualified_assets


,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,6.355086e+09,18.0,20.170000,4.205317e+08,3.151257e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,1.129218e+08,290.0,21.370000,5.285000e+06,5.285000e+06,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,9.762779e+07,318.0,0.292862,4.731356e+08,3.345974e+08,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi,BENQI,QI,4.018029e+07,561.0,0.011651,7.200000e+09,3.448925e+09,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5
avalaunch,Avalaunch,XAVA,1.715649e+07,839.0,0.535362,1.000000e+08,3.204097e+07,0xd1c3f94de7e5b45fa4edbba472491a9f4b166fc4


### Marketcap ranking & filtering

In [44]:
chosen_tokens = qualified_assets.sort_values("market_cap", ascending=False)

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 50%.
* All excess weight is proportionally redistributed to all uncapped tokens. 
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [45]:
# exclude savax from weighting since it should be a part of avax weight
if 'benqi-liquid-staked-avax' in chosen_tokens.index:
    chosen_tokens_ex_savax = chosen_tokens.drop('benqi-liquid-staked-avax')
    weights = chosen_tokens_ex_savax['market_cap'].div(chosen_tokens_ex_savax['market_cap'].sum())
else:
    weights = chosen_tokens['market_cap'].div(chosen_tokens['market_cap'].sum())



### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [46]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum()
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum()).mul(c)
    
    return w

In [47]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum())
    
    return w

In [48]:
adjusted_weights = adjust_weights(weights, max_weight=max_weight) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=min_weight)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=max_weight)

### Split AVAX 

In [49]:
if 'avalanche-2' and 'benqi-liquid-staked-avax' in chosen_tokens.index:
    
    adjusted_weights['benqi-liquid-staked-avax'] = adjusted_weights['avalanche-2'] * avax_to_savax_split
    adjusted_weights['avalanche-2'] = adjusted_weights['avalanche-2'] * (1 - avax_to_savax_split)

adjusted_weights =  adjusted_weights.sort_values(ascending = False)


### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [50]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<128]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [51]:
converted_weights = convert_weights(adjusted_weights)

## Summary table

In [52]:
cai = pd.DataFrame()
cai.index = chosen_tokens.index
cai['name'] = chosen_tokens['name']
cai['symbol'] = chosen_tokens['symbol']
cai['market_cap'] = chosen_tokens['market_cap']
cai['price'] = chosen_tokens['current_price']
cai['weight'] = adjusted_weights
cai['weight_converted'] = converted_weights
cai['address'] = chosen_tokens['addr']

cai = cai[cai['weight'] > 0]
cai = cai.sort_values("market_cap", ascending=False)
cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,6.355086e+09,20.170000,0.200000,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,1.129218e+08,21.370000,0.300000,76,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,9.762779e+07,0.292862,0.315000,81,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi,BENQI,QI,4.018029e+07,0.011651,0.129643,33,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5
avalaunch,Avalaunch,XAVA,1.715649e+07,0.535362,0.055356,14,0xd1c3f94de7e5b45fa4edbba472491a9f4b166fc4


In [53]:
# Add assets and weights to database
db_assets = db_funcs.convert_to_sql_strings(list(cai.index))
db_weights = list(cai['weight'])
db_funcs.insert_values(db_key_date,db_assets,db_weights,db_benchmark_table)

In [54]:
#Querying current contract to ascertain which assets will be dropped

cai_token_address = '0x48f88A3fE843ccb0b5003e70B4192c1d7448bEf0'
ia_contract = web3_object.eth.contract(address=cai_token_address,abi=index_anatomy)
addresses, weights = ia_contract.functions.anatomy().call()


for address in addresses:
    address = address.lower()
    if address not in list(cai['address']):
        data = tokens_market_data[tokens_market_data['addr']== address]
        cai.loc[data.index[0]] = [data['name'][0],data['symbol'][0],data['market_cap'][0],data['current_price'][0],0,0,address]

cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,6.355086e+09,20.170000,0.200000,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,1.129218e+08,21.370000,0.300000,76,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,9.762779e+07,0.292862,0.315000,81,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi,BENQI,QI,4.018029e+07,0.011651,0.129643,33,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5
avalaunch,Avalaunch,XAVA,1.715649e+07,0.535362,0.055356,14,0xd1c3f94de7e5b45fa4edbba472491a9f4b166fc4


In [55]:
zipped_assets = list(zip(list(cai['address']),list(cai['weight_converted'])))
sorted_assets = sorted(zipped_assets,key = lambda x: int(x[0],base=0) )

asset_string = ''
for i in sorted_assets:
    asset_string += f'{i[0]},'
asset_string = asset_string[:-1]
print(asset_string)
print([x[1] for x in sorted_assets])

0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,0xd1c3f94de7e5b45fa4edbba472491a9f4b166fc4
[76, 81, 33, 51, 14]
